In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
filename = '/content/drive/MyDrive/bigdata_analysis_final_project - movie recommender system(영화 추천 시스템)/Data Preprocessing/movie_dataset.csv'
movies_df = pd.read_csv('/content/drive/MyDrive/bigdata_analysis_final_project - movie recommender system(영화 추천 시스템)/Data Preprocessing/movie_dataset.csv')
movies_df.head()

,movieId,naver_code,title,title_kor,genres,keyword,director,cast,director_kor,cast_kor,...,naver_user_participate,naver_critics_rate,naver_critics_participate,nation,release_date,running_time,age,story,poster_url,url
0,1,17153,Toy Story,토이 스토리,Adventure Animation Children Comedy Fantasy,"jealousy,toy,boy,friendship,friends,rivalry,bo...",John Lasseter,"Tom Hanks,Tim Allen,Don Rickles",존 라세터,"톰 행크스, 팀 알렌",...,1393,0.0,0,미국,2010.05.05 1995.12.30,77분,"전체 관람가, G",우디(톰 행크스 목소리 분)는 6살짜리 남자 아이 앤디가 가장 아끼는 카우보이 인형...,https://movie-phinf.pstatic.net/20111222_26/13...,https://movie.naver.com/movie/bi/mi/basic.nave...
1,2,17440,Jumanji,쥬만지,Adventure Children Fantasy,"board game,disappearance,based on children's b...",Joe Johnston,"Robin Williams,Jonathan Hyde,Kirsten Dunst",조 존스톤,"로빈 윌리엄스, 커스틴 던스트, 데이빗 알란 그리어",...,1591,0.0,0,미국,1996.01.20,104분,"전체 관람가, PG",1969년. 커다란 공장을 운영하는 아버지를 둔 12세 소년 알랜 패리쉬(Young...,https://movie-phinf.pstatic.net/20111223_209/1...,https://movie.naver.com/movie/bi/mi/basic.nave...
2,4,17798,Waiting To Exhale,사랑을 기다리며,Comedy Drama Romance,"based on novel,interracial relationship,single...",Forest Whitaker,"Whitney Houston,Angela Bassett,Loretta Devine",포레스트 휘태커,"휘트니 휴스턴, 안젤라 바셋, 로레타 드바인",...,17,0.0,0,미국,1996.04.05,127분,"청소년 관람불가, R",사반나 잭슨(Savannah Jackson: 휘트니 휴스턴 분)은 TV 프로듀서로서...,https://movie-phinf.pstatic.net/20111221_192/1...,https://movie.naver.com/movie/bi/mi/basic.nave...
3,5,17754,Father Of The Bride Part II,신부의 아버지 2,Comedy,"baby,midlife crisis,confidence,aging,daughter,...",Charles Shyer,"Steve Martin,Diane Keaton,Martin Short",찰스 샤이어,"스티브 마틴, 다이안 키튼, 마틴 숏",...,13,0.0,0,미국,1996.05.04,106분,"15세 관람가, PG","사랑하는 딸을 시집보내고 난 뒤, 한동안 허전함을 극복하지 못했던 '신부의 아버지'...",https://movie-phinf.pstatic.net/20111222_214/1...,https://movie.naver.com/movie/bi/mi/basic.nave...
4,6,17327,Heat,히트,Action Crime Thriller,"robbery,detective,bank,obsession,chase,shootin...",Michael Mann,"Al Pacino,Robert De Niro,Val Kilmer",마이클 만,"알 파치노, 로버트 드 니로",...,1454,8.0,1,미국,2017.11.09 1996.08.10,170분,"청소년 관람불가, R",LA 경찰국 강력계의 빈센트 한나(알 파치노) 반장은 두 번의 이혼 경력과 순탄치 ...,https://movie-phinf.pstatic.net/20171024_181/1...,https://movie.naver.com/movie/bi/mi/basic.nave...


In [3]:
movies_df['director'] = movies_df['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
movies_df['director'] = movies_df['director'].apply(lambda x: [x, x, x])
movies_df['director']

0              [johnlasseter, johnlasseter, johnlasseter]
1                 [joejohnston, joejohnston, joejohnston]
2        [forestwhitaker, forestwhitaker, forestwhitaker]
3              [charlesshyer, charlesshyer, charlesshyer]
4                 [michaelmann, michaelmann, michaelmann]
                              ...                        
3959     [frédéricvideau, frédéricvideau, frédéricvideau]
3960    [timothywoodwardjr., timothywoodwardjr., timot...
3961                 [jiazhangke, jiazhangke, jiazhangke]
3962              [maxmakowski, maxmakowski, maxmakowski]
3963    [olivianeergaard-holm, olivianeergaard-holm, o...
Name: director, Length: 3964, dtype: object

In [4]:
movies_df['cast'] = movies_df['cast'].astype('str').apply(lambda x: [str.lower(x.replace(" ", ""))])
# movies_df['cast'] = movies_df['cast'].str.split(',')
movies_df['cast']

0                     [tomhanks,timallen,donrickles]
1          [robinwilliams,jonathanhyde,kirstendunst]
2       [whitneyhouston,angelabassett,lorettadevine]
3              [stevemartin,dianekeaton,martinshort]
4                  [alpacino,robertdeniro,valkilmer]
                            ...                     
3959      [agathebonitzer,redakateb,hélènefillières]
3960           [kaiwilyman,brucedern,deniserichards]
3961             [hongweiwang,haohongjian,baitaozuo]
3962        [nickstahl,eddiekayethomas,januaryjones]
3963                                    [davidlynch]
Name: cast, Length: 3964, dtype: object

In [5]:
movies_df['dc'] = movies_df['director'] + movies_df['cast']
movies_df['dc'] = movies_df['dc'].apply(lambda x: ' '.join(x))
movies_df[['dc']][:5]

,dc
0,johnlasseter johnlasseter johnlasseter tomhank...
1,joejohnston joejohnston joejohnston robinwilli...
2,forestwhitaker forestwhitaker forestwhitaker w...
3,charlesshyer charlesshyer charlesshyer stevema...
4,"michaelmann michaelmann michaelmann alpacino,r..."


In [6]:
# 개인 맞춤 장르요소 새로운 행추가
# 213 -> 'j.j.abrams rolandemmerich jonfavreau kevinreynolds lenwiseman nicolascage robertdowneyjr. zoesaldana chrispine tommyleejones'
# 475 -> 'user288_taste' ,'dc' : 'quentintarantino martinscorsese stevenspielberg davidyates j.j.abrams tomhanks bradpitt. mattdamon danielradcliffe emmawatson'
# 288 -> 'stevenspielberg ivanreitman bradsilberling christophercolumbus geoffmurphy tomhanks arnoldschwarzenegger caryelwes hughgrant jimcarrey'
# 64 -> 'joelschumacher edwardzwic kevincostner quentintarantino davidcarson johntravolta bradpitt. kevincostner valkilmer anthonyhopkins'
new_data = {'title': 'user64_taste' ,'dc' : 'joelschumacher edwardzwic kevincostner quentintarantino davidcarson johntravolta bradpitt. kevincostner valkilmer anthonyhopkins'}
movies_df = movies_df.append(new_data, ignore_index=True)

In [7]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
dc_mat = count_vect.fit_transform(movies_df['dc'])
print(dc_mat.shape)

(3965, 22436)


In [8]:
dc_sim = cosine_similarity(dc_mat, dc_mat)
print(dc_sim.shape)
print(dc_sim[:5])

(3965, 3965)
[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.05006262]]


In [9]:
dc_sim.argsort()[:3]

array([[1982, 2636, 2637, ..., 1507,  561,    0],
       [   0, 2634, 2635, ..., 2128, 1151,    1],
       [   0, 2632, 2633, ..., 2869,  402,    2]])

In [10]:
dc_sim_sorted_ind = dc_sim.argsort()[:, ::-1]
print(dc_sim_sorted_ind[:3])

[[   0  561 1507 ... 2637 2636 1982]
 [   1 1151 2128 ... 2635 2634    0]
 [   2  402 2869 ... 2633 2632    0]]


In [11]:
percentile = 0.6
m = movies_df['naver_user_participate'].quantile(percentile)
C = movies_df['naver_user_rate'].mean()

def weighted_rating_average(record):
  v = record['naver_user_participate']
  R = record['naver_user_rate']
  return ((v/(v+m))*R) + ((m/(m+v))*C)
  
movies_df['weighted_rate'] = movies_df.apply(weighted_rating_average, axis=1)
movies_df[['title','naver_user_rate','weighted_rate','naver_user_participate']].sort_values('weighted_rate', ascending=False)[:10]

,title,naver_user_rate,weighted_rate,naver_user_participate
3841,Band Of Brothers,9.73,9.410353,1966.0
373,Titanic,9.41,9.383097,23052.0
195,Terminator 2: Judgment Day,9.45,9.368184,7525.0
117,Forrest Gump,9.42,9.347925,8457.0
1935,3 Idiots,9.35,9.326742,25932.0
1991,Toy Story 3,9.40,9.321688,7682.0
3481,Be With You,9.35,9.311933,15726.0
420,Saving Private Ryan,9.43,9.306090,4817.0
279,Dead Poets Society,9.38,9.300176,7455.0
192,Home Alone,9.43,9.296718,4457.0


In [12]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
  title_movie = df[df['title'] == title_name]
  title_index = title_movie.index.values
    
  similar_indexes = sorted_ind[title_index, :(top_n*2)]
  similar_indexes = similar_indexes.reshape(-1)

  similar_indexes = similar_indexes[similar_indexes != title_index]

  result_df = df.iloc[similar_indexes].sort_values('weighted_rate', ascending=False)[:top_n]
  result_df = result_df[['movieId']]
  result_df.reset_index(drop=True, inplace=True)
  result_df.reset_index(drop=False, inplace=True)
  result_df.rename(columns={'index':'rank'}, inplace=True)
  result_df['rank'] = result_df['rank'] + 1
    
  return result_df

In [13]:
similar_movies = find_sim_movie(movies_df, dc_sim_sorted_ind, 'user64_taste', 5)
similar_movies

,rank,movieId
0,1,590.0
1,2,296.0
2,3,99114.0
3,4,68157.0
4,5,1089.0


In [14]:
filename = '/content/drive/MyDrive/bigdata_analysis_final_project - movie recommender system(영화 추천 시스템)/Data Preprocessing/movie_dataset.csv'
movies_dataset_df = pd.read_csv('/content/drive/MyDrive/bigdata_analysis_final_project - movie recommender system(영화 추천 시스템)/Data Preprocessing/movie_dataset.csv')
movies_dataset_df = movies_dataset_df[['title', 'title_kor', 'movieId', 'director', 'cast', 'poster_url']]
movies_dataset_df

,title,title_kor,movieId,director,cast,poster_url
0,Toy Story,토이 스토리,1,John Lasseter,"Tom Hanks,Tim Allen,Don Rickles",https://movie-phinf.pstatic.net/20111222_26/13...
1,Jumanji,쥬만지,2,Joe Johnston,"Robin Williams,Jonathan Hyde,Kirsten Dunst",https://movie-phinf.pstatic.net/20111223_209/1...
2,Waiting To Exhale,사랑을 기다리며,4,Forest Whitaker,"Whitney Houston,Angela Bassett,Loretta Devine",https://movie-phinf.pstatic.net/20111221_192/1...
3,Father Of The Bride Part II,신부의 아버지 2,5,Charles Shyer,"Steve Martin,Diane Keaton,Martin Short",https://movie-phinf.pstatic.net/20111222_214/1...
4,Heat,히트,6,Michael Mann,"Al Pacino,Robert De Niro,Val Kilmer",https://movie-phinf.pstatic.net/20171024_181/1...
...,...,...,...,...,...,...
3959,Coming Home,5일의 마중,175897,Frédéric Videau,"Agathe Bonitzer,Reda Kateb,Hélène Fillières",https://movie-phinf.pstatic.net/20140919_189/1...
3960,American Violence,폭력의 근원,175979,Timothy Woodward Jr.,"Kaiwi Lyman,Bruce Dern,Denise Richards",https://movie-phinf.pstatic.net/20170918_157/1...
3961,Pickpocket,소무,176175,Jia Zhangke,"Hongwei Wang,Hao Hongjian,Baitao Zuo",https://movie-phinf.pstatic.net/20111222_152/1...
3962,Taboo,고하토,183123,Max Makowski,"Nick Stahl,Eddie Kaye Thomas,January Jones",https://movie-phinf.pstatic.net/20111222_238/1...


In [15]:
recomm_movies = pd.merge(similar_movies, movies_dataset_df, how = 'inner', on=['movieId'])
recomm_movies[['rank', 'movieId', 'title', 'title_kor', 'director', 'cast', 'poster_url']]

,rank,movieId,title,title_kor,director,cast,poster_url
0,1,590.0,Dances With Wolves,늑대와 춤을,Kevin Costner,"Kevin Costner,Mary McDonnell,Graham Greene",https://movie-phinf.pstatic.net/20210106_266/1...
1,2,296.0,Pulp Fiction,펄프 픽션,Quentin Tarantino,"John Travolta,Samuel L. Jackson,Uma Thurman",https://movie-phinf.pstatic.net/20111221_33/13...
2,3,99114.0,Django Unchained,장고:분노의 추적자,Quentin Tarantino,"Jamie Foxx,Christoph Waltz,Leonardo DiCaprio",https://movie-phinf.pstatic.net/20130204_279/1...
3,4,68157.0,Inglourious Basterds,바스터즈: 거친 녀석들,Quentin Tarantino,"Brad Pitt,Mélanie Laurent,Christoph Waltz",https://movie-phinf.pstatic.net/20111223_87/13...
4,5,1089.0,Reservoir Dogs,저수지의 개들,Quentin Tarantino,"Harvey Keitel,Tim Roth,Michael Madsen",https://movie-phinf.pstatic.net/20111222_282/1...


In [16]:
recomm_movies.to_csv("contents_based_filtering_list(director&cast).csv", mode='w', index=False)

### end of file